# Vorlesung: Clustering von Text-Daten

Eingelesen wird eine Liste mit Beschreibungen von Kameras. Für die Daten zu Typ, Format, Baujahr und Objektiv-Typ, soll ein Merkmals-Vektor erzeugt werden. 

In [52]:
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.manifold import TSNE  
from sklearn.decomposition import TruncatedSVD
import matplotlib.pyplot as plt

import hdbscan

In [49]:
%matplotlib inline

In [50]:
dfKameras= pd.read_excel('KameraKatalog.xls')

In [51]:
dfKameras.dropna(axis=1,how='all').head()

,Katalog-Nr.,Modellbezeichnung,Typ,Format,Baujahr,Baujahrende,Objektiv,Wechselobjektiv,Objektivanschluss,Verschluß,Verschlußbezeichnung,Verschlusszeit,Erläuterungen,US$,Unnamed: 43,Unnamed: 46
0,SPZ0200,Bravolette Nr. 6580,Rollfilm,6x9,1908.0,NaN,Periskop,NaN,NaN,Einfach-Verschluss,Einfach-Verschluss,NaN,"Met.-Box-Kamera, wie vor.",94,NaN,NaN
1,KOD2740,Flush Back Kodak No.3,Rollfilm,3 1/4x4 1/4,1909.0,1915.0,Rapid Rectilinear,NaN,NaN,B&L Auto,B&L Automat,NaN,"Box-Falt-Kam, horiz. 48 Aufn, Platten od. Roll...",585,NaN,NaN
2,BRU0055,Paris Mod. II,Platten,13x18,1837.0,NaN,WO,Wechselobjektiv,NaN,NaN,NaN,NaN,"Nußbaum, dopp. Ausz., kon.Balgen, Messing, Zah...",293,NaN,NaN
3,STH0050,Daguerreotypie-Kamera,Daguerreotypie,8x11,1839.0,NaN,Steinheil,NaN,NaN,NaN,NaN,NaN,"Holz-Box-Kamera f0r Rollfilm, Geheimkamera ...",749,NaN,NaN
4,CHE0005,Chevalier Daguerreotypie,Platten,"8,2x19,8",1840.0,NaN,Chevalier Achromat,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Um den Dataframe zu reduzieren, wählen wir die gewünschten Spalten aus.

In [7]:
dfKamRed = dfKameras[['Typ','Format','Baujahr','Objektiv']]

In [8]:
dfKamRed.head()

,Typ,Format,Baujahr,Objektiv
0,Rollfilm,6x9,1908.0,Periskop
1,Rollfilm,3 1/4x4 1/4,1909.0,Rapid Rectilinear
2,Platten,13x18,1837.0,WO
3,Daguerreotypie,8x11,1839.0,Steinheil
4,Platten,"8,2x19,8",1840.0,Chevalier Achromat


In [64]:
dfKam1900 = dfKamRed[dfKamRed['Baujahr'] < 1900.0]

## Erzeugen des Wort Vektors

Das Paket scikit-learn bietet ein für unsere Zwecke passendes Tool: den CountVectorizer. Hiermit werden Listen von Sätzen nach den enthaltenen Wörtern in Zahlenreihen umgewandelt. Jeder Satz wird dann zu einer Zeile einer Matrix.

In [55]:
vectorizer = CountVectorizer(max_features=300)

Um die Funktion nutzen zu können, müssen wir erst NaN Einträge entfernen und alle Einträge in Strings umwandeln.

In [65]:
dfKam = dfKam1900.dropna()

In [66]:
types = dfKam['Typ'].apply(str).values 
formats = dfKam['Format'].apply(str).values
baujahre = dfKam['Baujahr'].apply(str).values
objektive = dfKam['Objektiv'].apply(str).values

Nun erzeugen wir künstlich Sätze, in dem wir für jede Zeile aus allen Einträgen der Spalten einen zusammenhängenden String bilden.

In [67]:
corpus = [a + ' ' + b + ' ' + c + ' '+d for a,b,c,d in zip(types,formats,baujahre,objektive)]

Aus dieser Satz-Liste können wir nun eine dünn besetzte Matrix bilden. Sonst würde durch die Menge der Datenpunkte der Speicherplatz nicht reichen. 

In [68]:
X = vectorizer.fit_transform(corpus)

In [69]:
X

<1204x300 sparse matrix of type '<class 'numpy.int64'>'
	with 5945 stored elements in Compressed Sparse Row format>

## Clustering

In [70]:
clusterer = hdbscan.HDBSCAN(metric='euclidean',min_cluster_size=5, min_samples=5, gen_min_span_tree=True)
clusterer.fit(X)

HDBSCAN(algorithm='best', allow_single_cluster=False, alpha=1.0,
    approx_min_span_tree=True, core_dist_n_jobs=4, gen_min_span_tree=True,
    leaf_size=40, match_reference_implementation=False,
    memory=Memory(cachedir=None), metric='euclidean', min_cluster_size=5,
    min_samples=5, p=None)